# 13. 태깅 작업(Tagging Task)

## 7) 문자 임베딩(Character Embedding) 활용하기

개체명 인식기의 성능을 올리기 위한 방법으로 문자 임베딩을 워드 임베딩과 함께 입력으로 사용하는 방법이 있습니다. 워드 임베딩에 문자 임베딩을 연결(concatenate)하여 성능을 높여봅시다.

### 1. 문자 임베딩(Char Embedding)을 위한 전처리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
data = pd.read_csv("datasets/ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")
data['Word'] = data['Word'].str.lower()

# 하나의 문장에 등장한 단어와 개체명 태깅 정보끼리 쌍(pair)으로 묶는 작업을 수행
func = lambda temp: [(w, t) for w, t in zip(temp["Word"].values.tolist(), temp["Tag"].values.tolist())]
tagged_sentences=[t for t in data.groupby("Sentence #").apply(func)]

# 각 순서에 등장하는 원소들끼리 묶어줍니다.
sentences, ner_tags = [], [] 
for tagged_sentence in tagged_sentences:
    sentence, tag_info = zip(*tagged_sentence) 
    sentences.append(list(sentence))
    ner_tags.append(list(tag_info))

In [3]:
# 모든 단어를 사용하며 인덱스 1에는 단어 'OOV'를 할당.
src_tokenizer = Tokenizer(oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)

# 태깅 정보들은 내부적으로 대문자를 유지한 채 저장
tar_tokenizer = Tokenizer(lower=False)
tar_tokenizer.fit_on_texts(ner_tags)

vocab_size = len(src_tokenizer.word_index) + 1
tag_size = len(tar_tokenizer.word_index) + 1
print('단어 집합의 크기 : {}'.format(vocab_size))
print('개체명 태깅 정보 집합의 크기 : {}'.format(tag_size))

X_data = src_tokenizer.texts_to_sequences(sentences)
y_data = tar_tokenizer.texts_to_sequences(ner_tags)

단어 집합의 크기 : 31819
개체명 태깅 정보 집합의 크기 : 18


In [4]:
word_to_index = src_tokenizer.word_index
index_to_word = src_tokenizer.index_word

ner_to_index = tar_tokenizer.word_index
index_to_ner = tar_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [5]:
max_len = 70
X_data = pad_sequences(X_data, padding='post', maxlen=max_len)
y_data = pad_sequences(y_data, padding='post', maxlen=max_len)

X_train, X_test, y_train_int, y_test_int = train_test_split(X_data, y_data, 
                                                            test_size=.2, 
                                                            random_state=777)

y_train = to_categorical(y_train_int, num_classes=tag_size)
y_test = to_categorical(y_test_int, num_classes=tag_size)

In [6]:
# char_vocab 만들기
words = list(set(data["Word"].values))
chars = set([w_i for w in words for w_i in w])
chars = sorted(list(chars))
print('문자 집합 :',chars)

문자 집합 : ['!', '"', '#', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '\x85', '\x91', '\x92', '\x93', '\x94', '\x96', '\x97', '\xa0', '°', 'é', 'ë', 'ö', 'ü']


이렇게 얻은 문자 집합으로부터 문자를 정수로 변환할 수 있는 딕셔너리인 char_to_index와 반대로 정수로부터 문자를 얻을 수 있는 딕셔너리인 index_to_char를 만듭니다.

In [7]:
char_to_index = {c: i + 2 for i, c in enumerate(chars)}
char_to_index["OOV"] = 1
char_to_index["PAD"] = 0

index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [8]:
max_len_char = 15

# 문자 시퀀스에 대한 패딩하는 함수
def padding_char_indice(char_indice, max_len_char):
  return pad_sequences(
        char_indice, maxlen=max_len_char, padding='post', value = 0)

# 각 단어를 문자 시퀀스로 변환 후 패딩 진행
def integer_coding(sentences):
  char_data = []
  for ts in sentences:
    word_indice = [word_to_index[t] for t in ts]
    char_indice = [[char_to_index[char] for char in t]  
                                          for t in ts]
    char_indice = padding_char_indice(char_indice, max_len_char)

    for chars_of_token in char_indice:
      if len(chars_of_token) > max_len_char:
        continue
    char_data.append(char_indice)
  return char_data

In [9]:
# 문자 단위 정수 인코딩 결과
X_char_data = integer_coding(sentences)

동일한 문장에 대해서 단어 단위 정수 인코딩과 문자 단위 정수 인코딩의 차이를 확인해봅시다. 

첫번째 샘플은 다음과 같습니다.

In [10]:
# 정수 인코딩 이전의 기존 문장
print('기존 문장 :')
print(sentences[0])

기존 문장 :
['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.']


위 문장을 정수 인코딩 및 패딩한 결과는 다음과 같습니다.

In [11]:
# 단어 단위 정수 인코딩 + 패딩
print('단어 단위 정수 인코딩 :')
print(X_data[0])

단어 단위 정수 인코딩 :
[ 254    6  967   16 1795  238  468    7  523    2  129    5   61    9
  571    2  833    6  186   90   22   15   56    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]


254는 기존의 thousands, 6은 기존의 of에 해당됩니다. 해당 샘플을 문자 단위 정수 인코딩한 결과는 다음과 같습니다.

In [12]:
# 문자 단위 정수 인코딩
print('문자 단위 정수 인코딩 :')
print(X_char_data[0])

문자 단위 정수 인코딩 :
[[53 41 48 54 52 34 47 37 52  0  0  0  0  0  0]
 [48 39  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [37 38 46 48 47 52 53 51 34 53 48 51 52  0  0]
 [41 34 55 38  0  0  0  0  0  0  0  0  0  0  0]
 [46 34 51 36 41 38 37  0  0  0  0  0  0  0  0]
 [53 41 51 48 54 40 41  0  0  0  0  0  0  0  0]
 [45 48 47 37 48 47  0  0  0  0  0  0  0  0  0]
 [53 48  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [49 51 48 53 38 52 53  0  0  0  0  0  0  0  0]
 [53 41 38  0  0  0  0  0  0  0  0  0  0  0  0]
 [56 34 51  0  0  0  0  0  0  0  0  0  0  0  0]
 [42 47  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [42 51 34 50  0  0  0  0  0  0  0  0  0  0  0]
 [34 47 37  0  0  0  0  0  0  0  0  0  0  0  0]
 [37 38 46 34 47 37  0  0  0  0  0  0  0  0  0]
 [53 41 38  0  0  0  0  0  0  0  0  0  0  0  0]
 [56 42 53 41 37 51 34 56 34 45  0  0  0  0  0]
 [48 39  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [35 51 42 53 42 52 41  0  0  0  0  0  0  0  0]
 [53 51 48 48 49 52  0  0  0  0  0  0  0  0  0]
 [39 51 48 46  0  0  0  0

위 출력 결과에서 각 행은 각 단어를 의미합니다. 가령, thousands는 첫번째 행 [53 41 48 54 52 34 47 37 52 0 0 0 0 0 0]에 해당됩니다. 

단어의 최대 길이를 15(max_len_char)로 제한하였으므로, 길이가 15보다 짧은 단어는 뒤에 0으로 패딩됩니다.

X_data는 뒤에 0으로 패딩되어 길이가 70인 것에 비해 X_char_data는 현재 0번 단어는 무시되어 길이가 70이 아닙니다. 다시 말해 위 출력 결과에서 행의 개수가 70이 아닌 상태입니다. 

길이 70으로 맞춰주기 위해서 문장 길이 방향으로도 패딩을 해줍니다.

In [13]:
X_char_data = pad_sequences(X_char_data, maxlen=max_len, 
                            padding='post', value=0)

In [14]:
X_char_train, X_char_test, _, _ = train_test_split(X_char_data, y_data, 
                                                   test_size=.2, 
                                                   random_state=777)

X_char_train = np.array(X_char_train)
X_char_test = np.array(X_char_test)

In [15]:
print(X_train[0])

[ 150  928  361   17 2624    9 4131 3567    9    8 2893 1250  880  107
    3    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [16]:
print(index_to_word[150])

soldiers


In [17]:
print(' '.join([index_to_char[index] for index in X_char_train[0][0]]))

s o l d i e r s PAD PAD PAD PAD PAD PAD PAD


In [18]:
print('훈련 샘플 문장의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('훈련 샘플 char 데이터의 크기 : {}'.format(X_char_train.shape))
print('테스트 샘플 문장의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))

훈련 샘플 문장의 크기 : (38367, 70)
훈련 샘플 레이블의 크기 : (38367, 70, 18)
훈련 샘플 char 데이터의 크기 : (38367, 70, 15)
테스트 샘플 문장의 크기 : (9592, 70)
테스트 샘플 레이블의 크기 : (9592, 70, 18)


### 2. BiLSTM-CNN을 이용한 개체명 인식

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, TimeDistributed, Dropout, concatenate, Bidirectional, LSTM, Conv1D, Dense, MaxPooling1D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from seqeval.metrics import f1_score, classification_report
from keras_crf import CRFModel

embedding_dim = 128
char_embedding_dim = 64
dropout_ratio = 0.5
hidden_units = 256
num_filters = 30
kernel_size = 3

# 단어 임베딩
word_ids = Input(shape=(None,),dtype='int32', name='words_input')
word_embeddings = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(word_ids)

# char 임베딩
char_ids = Input(shape=(None, max_len_char,), name='char_input')
embed_char_out = TimeDistributed(Embedding(len(char_to_index), char_embedding_dim, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(char_ids)
dropout = Dropout(dropout_ratio)(embed_char_out)

# char 임베딩에 대해서는 Conv1D 수행
conv1d_out = TimeDistributed(Conv1D(kernel_size=kernel_size, filters=num_filters, 
                                    padding='same', activation='tanh', 
                                    strides=1))(dropout)
maxpool_out = TimeDistributed(MaxPooling1D(max_len_char))(conv1d_out)
char_embeddings = TimeDistributed(Flatten())(maxpool_out)
char_embeddings = Dropout(dropout_ratio)(char_embeddings)

# char 임베딩을 Conv1D 수행한 뒤에 단어 임베딩과 연결
output = concatenate([word_embeddings, char_embeddings])

# 연결한 벡터를 가지고 문장의 길이만큼 LSTM을 수행
output = Bidirectional(LSTM(hidden_units, return_sequences=True, dropout=dropout_ratio))(output)

# 출력층
output = TimeDistributed(Dense(tag_size, activation='softmax'))(output)

model = Model(inputs=[word_ids, char_ids], outputs=[output])
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('results/bilstm_cnn_1307.h5', monitor='val_acc', mode='max', 
                     verbose=1, save_best_only=True)

# history = model.fit([X_train, X_char_train], y_train, batch_size=128, 
#                     epochs=15, validation_split=0.1, verbose=1, 
#                     callbacks=[es, mc])

Epoch 1/15
270/270 [==============================] - 24s 62ms/step - loss: 0.2080 - acc: 0.9470 - val_loss: 0.0901 - val_acc: 0.9753

Epoch 00001: val_acc improved from -inf to 0.97526, saving model to results\bilstm_cnn_1307.h5
Epoch 2/15
270/270 [==============================] - 16s 59ms/step - loss: 0.0656 - acc: 0.9814 - val_loss: 0.0523 - val_acc: 0.9847

Epoch 00002: val_acc improved from 0.97526 to 0.98474, saving model to results\bilstm_cnn_1307.h5
Epoch 3/15
270/270 [==============================] - 16s 60ms/step - loss: 0.0457 - acc: 0.9867 - val_loss: 0.0464 - val_acc: 0.9861

Epoch 00003: val_acc improved from 0.98474 to 0.98609, saving model to results\bilstm_cnn_1307.h5
Epoch 4/15
270/270 [==============================] - 16s 60ms/step - loss: 0.0381 - acc: 0.9887 - val_loss: 0.0422 - val_acc: 0.9872

Epoch 00004: val_acc improved from 0.98609 to 0.98715, saving model to results\bilstm_cnn_1307.h5
Epoch 5/15
270/270 [==============================] - 16s 59ms/step - l

In [21]:
model = load_model('results/bilstm_cnn_1307.h5')

테스트 데이터의 13번 인덱스의 샘플에 대해서 예측해봅시다.

In [22]:
i = 13 # 확인하고 싶은 테스트용 샘플의 인덱스.
# 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = model.predict([np.array([X_test[i]]), np.array([X_char_test[i]])])

y_predicted = np.argmax(y_predicted, axis=-1) # 확률 벡터를 정수 인코딩으로 변경.
labels = np.argmax(y_test[i], -1) # 원-핫 인코딩을 정수 인코딩으로 변경.

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for word, tag, pred in zip(X_test[i], labels, y_predicted[0]):
    if word != 0: # PAD값은 제외함.
        print("{:17}: {:7} {}".format(index_to_word[word], 
                                      index_to_ner[tag], 
                                      index_to_ner[pred]))

단어             |실제값  |예측값
-----------------------------------
the              : O       O
statement        : O       O
came             : O       O
as               : O       O
u.n.             : B-org   B-org
secretary-general: I-org   I-org
kofi             : B-per   B-per
annan            : I-per   I-per
met              : O       O
with             : O       O
officials        : O       O
in               : O       O
amman            : B-geo   B-geo
to               : O       O
discuss          : O       O
wednesday        : B-tim   B-tim
's               : O       O
attacks          : O       O
.                : O       O


In [24]:
def sequences_to_tag(sequences):
    result = []
    # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
    for sequence in sequences:
        word_sequence = []
        # 시퀀스로부터 확률 벡터 또는 원-핫 벡터를 하나씩 꺼낸다.
        for pred in sequence:
            # 정수로 변환. 예를 들어 pred가 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            pred_index = np.argmax(pred)            
            # index_to_ner을 사용하여 정수를 태깅 정보로 변환. 'PAD'는 'O'로 변경.
            word_sequence.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(word_sequence)
    return result

def sequences_to_tag_for_crf(sequences): 
    result = []
    # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
    for sequence in sequences: 
        word_sequence = []
        # 시퀀스로부터 예측 정수 레이블을 하나씩 꺼낸다.
        for pred_index in sequence:
            # index_to_ner을 사용하여 정수를 태깅 정보로 변환. 'PAD'는 'O'로 변경.
            word_sequence.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(word_sequence)
    return result

예측값과 실제값에 대한 태깅 정보 시퀀스를 얻은 후 F1-score를 계산합니다.

In [25]:
y_predicted = model.predict([X_test, X_char_test])
pred_tags = sequences_to_tag(y_predicted)
test_tags = sequences_to_tag(y_test)

print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))
print(classification_report(test_tags, pred_tags))

F1-score: 79.1%


C:\Users\seungwon\anaconda3\envs\scratch\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         art       0.00      0.00      0.00        63
         eve       0.92      0.21      0.34        52
         geo       0.84      0.84      0.84      7620
         gpe       0.95      0.94      0.95      3145
         nat       0.00      0.00      0.00        37
         org       0.62      0.55      0.58      4033
         per       0.72      0.75      0.73      3545
         tim       0.87      0.82      0.85      4067

   micro avg       0.80      0.78      0.79     22562
   macro avg       0.61      0.51      0.54     22562
weighted avg       0.80      0.78      0.79     22562



### 3. BiLSTM-CNN-CRF

In [27]:
embedding_dim = 128
char_embedding_dim = 64
dropout_ratio = 0.5
hidden_units = 256
num_filters = 30
kernel_size = 3

# 단어 임베딩
word_ids = Input(shape=(None,),dtype='int32')
word_embeddings = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(word_ids)

# char 임베딩
char_ids = Input(shape=(None, max_len_char,))
embed_char_out = TimeDistributed(Embedding(len(char_to_index), char_embedding_dim, 
                                           embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)))(char_ids)
dropout = Dropout(dropout_ratio)(embed_char_out)

# char 임베딩에 대해서는 Conv1D 수행
conv1d_out = TimeDistributed(Conv1D(kernel_size=kernel_size, filters=num_filters, 
                                    padding='same', activation='tanh', 
                                    strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(max_len_char))(conv1d_out)
char_embeddings = TimeDistributed(Flatten())(maxpool_out)
char_embeddings = Dropout(dropout_ratio)(char_embeddings)

# char 임베딩을 Conv1D 수행한 뒤에 단어 임베딩과 연결
output = concatenate([word_embeddings, char_embeddings])

# 연결한 벡터를 가지고 문장의 길이만큼 LSTM을 수행
output = Bidirectional(LSTM(hidden_units, return_sequences=True, 
                            dropout=dropout_ratio))(output)

# 출력층
output = TimeDistributed(Dense(tag_size, activation='relu'))(output)

base = Model(inputs=[word_ids, char_ids], outputs=[output])
model = CRFModel(base, tag_size)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), metrics='accuracy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('results/bilstm_cnn_crf_1307/cp.ckpt', 
                     monitor='val_decode_sequence_accuracy', 
                     mode='max', verbose=1, save_best_only=True, 
                     save_weights_only=True)

# history = model.fit([X_train, X_char_train], y_train_int, batch_size=128, 
#                     epochs=15, validation_split=0.1, 
#                     callbacks=[mc, es])

Epoch 1/15
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
270/270 [==============================] - 78s 274ms/step - decode_sequence_accuracy: 0.9400 - loss: 19.4026 - val_decode_sequence_accuracy: 0.9668 - val_loss: 8.0164

Epoch 00001: val_decode_sequence_accuracy improved from -inf to 0.96683, saving model to results/bilstm_cnn_crf_1307\cp.ckpt
Epoch 2/15
270/270 [==============================] - 72s 268ms/step - decode_sequence_accuracy: 0.9787 - loss: 5.0440 - val_decode_sequence_accuracy: 0.9834 - val_loss: 3.7907

Epoch 00002: val_decode_sequence_accuracy improved from 0.96683 to 0.98342, saving model to results/bilstm_cnn_crf_1307\cp.ckpt
Epoch 3/15
270/270 [==============================] - 73s 272ms/step - decode_sequence_accuracy: 0.9860 - loss: 2.9955 - val_decode_sequence_accuracy: 0.9851 - val_loss: 3.0915

Epoch 00003: val_decode_sequence_accuracy improved from 0.98342 to 0.98508

In [28]:
model.load_weights('results/bilstm_cnn_crf_1307/cp.ckpt')

테스트 데이터의 13번 인덱스의 샘플에 대해서 예측해봅시다.

In [29]:
i = 13 # 확인하고 싶은 테스트용 샘플의 인덱스.
# 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = model.predict([np.array([X_test[i]]), np.array([X_char_test[i]])])[0] 
labels = np.argmax(y_test[i], -1) # 원-핫 벡터를 정수 인코딩으로 변경.

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for word, tag, pred in zip(X_test[i], labels, y_predicted[0]):
    if word != 0: # PAD값은 제외함.
        print("{:17}: {:7} {}".format(index_to_word[word], 
                                      index_to_ner[tag], 
                                      index_to_ner[pred]))

단어             |실제값  |예측값
-----------------------------------
the              : O       O
statement        : O       O
came             : O       O
as               : O       O
u.n.             : B-org   B-org
secretary-general: I-org   I-org
kofi             : B-per   B-per
annan            : I-per   I-per
met              : O       O
with             : O       O
officials        : O       O
in               : O       O
amman            : B-geo   B-geo
to               : O       O
discuss          : O       O
wednesday        : B-tim   B-tim
's               : O       O
attacks          : O       O
.                : O       O


예측값과 실제값에 대한 태깅 정보 시퀀스를 얻은 후 F1-score를 계산합니다.

In [30]:
y_predicted = model.predict([X_test, X_char_test])[0]
pred_tags = sequences_to_tag_for_crf(y_predicted)
test_tags = sequences_to_tag(y_test)

print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))
print(classification_report(test_tags, pred_tags))

F1-score: 80.5%
              precision    recall  f1-score   support

         art       0.00      0.00      0.00        63
         eve       0.00      0.00      0.00        52
         geo       0.80      0.88      0.84      7620
         gpe       0.95      0.94      0.94      3145
         nat       0.00      0.00      0.00        37
         org       0.70      0.54      0.61      4033
         per       0.76      0.75      0.75      3545
         tim       0.88      0.85      0.86      4067

   micro avg       0.82      0.79      0.80     22562
   macro avg       0.51      0.49      0.50     22562
weighted avg       0.81      0.79      0.80     22562



### 4. BiLSTM-BiLSTM-CRF

In [31]:
embedding_dim = 128
char_embedding_dim = 64
dropout_ratio = 0.3
hidden_units = 64

# 단어 임베딩
word_ids = Input(batch_shape=(None, None), dtype='int32')
word_embeddings = Embedding(input_dim=vocab_size,
                                        output_dim=embedding_dim,
                                        name='word_embedding')(word_ids)

# char 임베딩
char_ids = Input(batch_shape=(None, None, None), dtype='int32')
char_embeddings = Embedding(input_dim=(len(char_to_index)), output_dim=char_embedding_dim,
                                        embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5),
                                        name='char_embedding')(char_ids)
char_embeddings = TimeDistributed(Bidirectional(LSTM(hidden_units)))(char_embeddings)

# char 임베딩을 BiLSTM을 통과 시켜 단어 벡터를 얻고 단어 임베딩과 연결
output = concatenate([word_embeddings, char_embeddings])

# 연결한 벡터를 가지고 문장의 길이만큼 LSTM을 수행
output = Dropout(dropout_ratio)(output)
output = Bidirectional(LSTM(units=hidden_units, return_sequences=True))(output)

# 출력층
output = TimeDistributed(Dense(tag_size, activation='relu'))(output)

base = Model(inputs=[word_ids, char_ids], outputs=[output])
model = CRFModel(base, tag_size)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), metrics='accuracy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('results/bilstm_bilstm_crf_1307/cp.ckpt', 
                     monitor='val_decode_sequence_accuracy', 
                     mode='max', verbose=1, save_best_only=True, 
                     save_weights_only=True)

# history = model.fit([X_train, X_char_train], y_train_int, batch_size=128, 
#                     epochs=15, validation_split=0.1, 
#                     callbacks=[mc, es])

Epoch 1/15
270/270 [==============================] - 84s 291ms/step - decode_sequence_accuracy: 0.9247 - loss: 24.5613 - val_decode_sequence_accuracy: 0.9708 - val_loss: 7.5368

Epoch 00001: val_decode_sequence_accuracy improved from -inf to 0.97080, saving model to results/bilstm_bilstm_crf_1307\cp.ckpt
Epoch 2/15
270/270 [==============================] - 76s 282ms/step - decode_sequence_accuracy: 0.9786 - loss: 5.1610 - val_decode_sequence_accuracy: 0.9811 - val_loss: 4.3224

Epoch 00002: val_decode_sequence_accuracy improved from 0.97080 to 0.98111, saving model to results/bilstm_bilstm_crf_1307\cp.ckpt
Epoch 3/15
270/270 [==============================] - 76s 282ms/step - decode_sequence_accuracy: 0.9843 - loss: 3.3460 - val_decode_sequence_accuracy: 0.9842 - val_loss: 3.3597

Epoch 00003: val_decode_sequence_accuracy improved from 0.98111 to 0.98415, saving model to results/bilstm_bilstm_crf_1307\cp.ckpt
Epoch 4/15
270/270 [==============================] - 77s 284ms/step - deco

In [32]:
model.load_weights('results/bilstm_bilstm_crf_1307/cp.ckpt')

테스트 데이터의 13번 인덱스의 샘플에 대해서 예측합니다.

In [33]:
i = 13 # 확인하고 싶은 테스트용 샘플의 인덱스.
# 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = model.predict([np.array([X_test[i]]), np.array([X_char_test[i]])])[0]
labels = np.argmax(y_test[i], -1) # 원-핫 벡터를 정수 인코딩으로 변경.

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for word, tag, pred in zip(X_test[i], labels, y_predicted[0]):
    if word != 0: # PAD값은 제외함.
        print("{:17}: {:7} {}".format(index_to_word[word], 
                                      index_to_ner[tag], 
                                      index_to_ner[pred]))

단어             |실제값  |예측값
-----------------------------------
the              : O       O
statement        : O       O
came             : O       O
as               : O       O
u.n.             : B-org   B-org
secretary-general: I-org   I-org
kofi             : B-per   B-per
annan            : I-per   I-per
met              : O       O
with             : O       O
officials        : O       O
in               : O       O
amman            : B-geo   B-geo
to               : O       O
discuss          : O       O
wednesday        : B-tim   B-tim
's               : O       O
attacks          : O       O
.                : O       O


예측값과 실제값에 대한 태깅 정보 시퀀스를 얻은 후 F1-score를 계산합니다.|

In [34]:
y_predicted = model.predict([X_test, X_char_test])[0]
pred_tags = sequences_to_tag_for_crf(y_predicted)
test_tags = sequences_to_tag(y_test)

print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))
print(classification_report(test_tags, pred_tags))

F1-score: 80.7%
              precision    recall  f1-score   support

         art       0.00      0.00      0.00        63
         eve       1.00      0.13      0.24        52
         geo       0.84      0.86      0.85      7620
         gpe       0.95      0.94      0.94      3145
         nat       0.00      0.00      0.00        37
         org       0.65      0.59      0.62      4033
         per       0.77      0.74      0.76      3545
         tim       0.89      0.83      0.86      4067

   micro avg       0.82      0.79      0.81     22562
   macro avg       0.64      0.51      0.53     22562
weighted avg       0.82      0.79      0.80     22562

